In [ ]:
# --- Step 1: Reload all our models and tokenizers ---
# We need these available in our new training notebook.
# This will be fast as it uses the cache.
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_checkpoints = {
    "BART": "facebook/bart-base",
    "T5": "t5-small",
    "PEGASUS": "google/pegasus-xsum"
}

tokenizers = {}
models = {}

print("--- Loading Models and Tokenizers ---")
for model_name, checkpoint in model_checkpoints.items():
    print(f"Loading {model_name}...")
    tokenizers[model_name] = AutoTokenizer.from_pretrained(checkpoint)
    models[model_name] = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# --- Step 2: Load our pre-processed datasets from disk ---
from datasets import load_from_disk, DatasetDict

tokenized_datasets = {}
print("\n--- Loading processed datasets from disk ---")
for model_name in model_checkpoints.keys():
    data_dir = f"./tokenized_data/{model_name}"
    print(f"Loading {model_name} dataset from {data_dir}")
    tokenized_datasets[model_name] = load_from_disk(data_dir)

print("\n--- All models, tokenizers, and datasets are ready! ---")
print(tokenized_datasets)

In [ ]:
import evaluate
import numpy as np

# Load the ROUGE metric from the 'evaluate' library
metric = evaluate.load("rouge")

print("ROUGE metric loaded.")

# This is a critical function that will be called during training
# It takes the model's predictions (eval_preds) and calculates the ROUGE score
def compute_metrics(eval_preds):
    # 'preds' are the raw logit outputs from the model
    # 'labels' are the true token IDs for the summary
    preds, labels = eval_preds

    # We need to convert the token IDs back into text
    # We use batch_decode for this
    # We skip special tokens (like <pad>) to get clean text
    decoded_preds = tokenizers["BART"].batch_decode(preds, skip_special_tokens=True)
    
    # We also need to decode the true labels
    # We replace -100 (which are padding tokens) with the pad_token_id
    # so they can be properly decoded
    labels = np.where(labels != -100, labels, tokenizers["BART"].pad_token_id)
    decoded_labels = tokenizers["BART"].batch_decode(labels, skip_special_tokens=True)
    
    # Add a newline after each sentence for the ROUGE metric
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    # Compute the ROUGE scores
    result = metric.compute(
        predictions=decoded_preds, 
        references=decoded_labels, 
        use_stemmer=True
    )
    
    # Extract the main scores
    result = {key: value * 100 for key, value in result.items()}
    
    # Add a 'gen_len' (generated length) metric
    prediction_lens = [np.count_nonzero(pred != tokenizers["BART"].pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

print("compute_metrics() function is defined and ready.")

In [ ]:
import torch
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# REMOVED: from transformers.training_args import IntervalStrategy 

# Check if CUDA is available (Expected to be False for the stable CPU path)
if torch.cuda.is_available():
    print("GPU (CUDA) is available! Training will be much faster.")
    use_fp16 = True 
else:
    print("No GPU (CUDA) found. Proceeding with stable CPU training.")
    use_fp16 = False 

print("Training libraries imported and variables set.")

In [ ]:
# Cell 4 in 03_Model_Training.ipynb - Launch Training

# A dictionary to store our final results from the training run
training_results = {}

print("--- Starting Comparative Model Training Loop (CPU) ---")

# Loop through each model in our dictionary
for model_name, model in models.items():
    print(f"\n=========================================")
    print(f"        Starting Training for {model_name}")
    print(f"=========================================")
    
    # Define the training configuration using stable string arguments
    args = Seq2SeqTrainingArguments(
        output_dir=f"./{model_name}-finetuned",
        
        # FINAL FIX: Using the shorter parameter name 'eval_strategy' for compatibility
        eval_strategy="epoch",  # Corrected parameter name
        save_strategy="epoch",                   
        
        # Core Hyperparameters
        learning_rate=2e-5,                      
        per_device_train_batch_size=4,           
        per_device_eval_batch_size=4,            
        num_train_epochs=3,                      # Train for 3 full passes
        weight_decay=0.01,                       
        save_total_limit=2,                      
        
        # CRITICAL for Summarization/Seq2Seq tasks
        predict_with_generate=True,              
        
        # Windows/CPU Stability Settings:
        fp16=False, # Explicitly disabling GPU mixed precision
        dataloader_num_workers=0, # Ensures stable multiprocessing on Windows CPU
        
        # Evaluation Settings
        load_best_model_at_end=True,
        metric_for_best_model="rouge2", # Use the ROUGE-2 score to determine the best checkpoint
    )
    
    # The Data Collator prepares batches dynamically during training
    data_collator = DataCollatorForSeq2Seq(
        tokenizer=tokenizers[model_name], 
        model=model
    )
    
    # The Trainer class manages the entire training and evaluation loop
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets[model_name]["train"],
        eval_dataset=tokenized_datasets[model_name]["validation"],
        data_collator=data_collator,
        tokenizer=tokenizers[model_name],
        compute_metrics=compute_metrics  # Our ROUGE scoring function
    )

    # --- Start Training! ---
    print(f"Training {model_name}...")
    result = trainer.train()
    
    # Store the results of the training run
    training_results[model_name] = result
    
    # --- Save the Final, Best Model (based on ROUGE-2) ---
    print(f"Saving best {model_name} model...")
    trainer.save_model(f"./{model_name}-best-finetuned")

    print(f"--- Finished with {model_name} ---")

print("\n--- !!! ALL TRAINING COMPLETE !!! ---")
print("Final results:", training_results)